# Práctica puertas cuánticas

El  objetivo  de  la  presente  actividad  es  entender  el  funcionamiento  de  la  superposición  y  las  puertas  de 
control.

In [95]:
from pyquil import get_qc, Program
from pyquil.gates import I, H, X, MEASURE
from pyquil.quilbase import Declare

In [96]:
def es_ganador(resultado, tiradas):
    jugadas_ganadas_j1 = 0
    jugadas_ganadas_j2 = 0
    empates = 0
    
    for i in resultado:
        # Jugador 1: cara(0), Jugador 2: cruz(1)
        if i[0] == 0 and i[1] == 1: 
            jugadas_ganadas_j1 += 1
        # Jugador 1: cruz(1), Jugador 2: cara(0)
        elif i[0] == 1 and i[1] == 0:  
            jugadas_ganadas_j2 += 1
        # Ambos sacan lo mismo
        else:  
            empates += 1
    
    print(f"Resultados después de {tiradas} tiradas:")
    print(f"  Jugador 1: {jugadas_ganadas_j1} victorias")
    print(f"  Jugador 2: {jugadas_ganadas_j2} victorias")
    print(f"  Empates: {empates}")
    print()
    
    if jugadas_ganadas_j1 > jugadas_ganadas_j2:
        print(f"Jugador 1 GANA con {jugadas_ganadas_j1} puntos")
    elif jugadas_ganadas_j2 > jugadas_ganadas_j1:
        print(f"Jugador 2 GANA con {jugadas_ganadas_j2} puntos")
    else:
        print(f"EMPATE: ambos con {jugadas_ganadas_j1} puntos")

## Parte 1

Simular el comportamiento aleatorio de ambos jugadores y comprobar quién vence en 50 tiradas. 

In [97]:
prog = Program(
    Declare('ro', 'BIT', 2),
    I(0), I(1),
    H(0), H(1),
    MEASURE(0, ('ro', 0)),
    MEASURE(1, ('ro', 1))
).wrap_in_numshots_loop(50)

qvm = get_qc('2q-qvm')
result = qvm.run(qvm.compile(prog)).get_register_map().get("ro")

es_ganador(
    resultado= result,
    tiradas=50
)

Resultados después de 50 tiradas:
  Jugador 1: 10 victorias
  Jugador 2: 20 victorias
  Empates: 20

Jugador 2 GANA con 20 puntos


## Parte 2

La segunda parte es donde el primer jugador será un tramposo. El segundo jugador debe seguir lanzando 
aleatoriamente la moneda, pero el primer jugador busca garantizarse ganar o empatar siempre, por lo que 
para su moneda debe salir siempre cara, partiendo de un estado inicial aleatorio. Buscamos construir el 
circuito y comprobar de nuevo quién vence en 50 tiradas (recordad, el segundo jugador no debería vencer 
nunca).


In [98]:
prog = Program(
    Declare('ro', 'BIT', 2),
    I(0),H(1),
    MEASURE(0, ('ro', 0)),  
    MEASURE(1, ('ro', 1))
).wrap_in_numshots_loop(50)

qvm = get_qc('2q-qvm')
result = qvm.run(qvm.compile(prog)).get_register_map().get("ro")

es_ganador(resultado=result, tiradas=50)

Resultados después de 50 tiradas:
  Jugador 1: 22 victorias
  Jugador 2: 0 victorias
  Empates: 28

Jugador 1 GANA con 22 puntos


## Parte 3

La última parte de la práctica incluye un tercer jugador. Este tercer jugador, que no está de acuerdo con las 
trampas realizadas en las partidas anteriores, busca el resultado contrario al segundo jugador, es decir, que 
siempre salga cruz. ¿Cómo puede garantizar este resultado partiendo del circuito resultante de la segunda 
parte?

In [ ]:
def es_ganador_3_jugadores(resultado, tiradas):
    victorias_j1 = 0
    empates = 0
    
    for i in resultado:        
        if i[0] == 0 and i[1] == 1 and i[2] == 1:  
            victorias_j1 += 1
        elif i[0] == 1 and i[1] == 0 and i[2] == 0:  
            victorias_j1 += 1
        elif i[1] == 0 and i[0] == 0 and i[2] == 1:  
            empates += 1  
        elif i[1] == 1 and i[0] == 0 and i[2] == 1:  
            empates += 1
        else:
            empates += 1
    
    # Contar por jugador
    j1_caras = sum([1 for i in resultado if i[0] == 0])
    j1_cruces = sum([1 for i in resultado if i[0] == 1])
    j2_caras = sum([1 for i in resultado if i[1] == 0])
    j2_cruces = sum([1 for i in resultado if i[1] == 1])
    j3_caras = sum([1 for i in resultado if i[2] == 0])
    j3_cruces = sum([1 for i in resultado if i[2] == 1])
    
    print(f"  Jugador 1: {j1_caras} caras, {j1_cruces} cruces")
    print(f"  Jugador 2: {j2_caras} caras, {j2_cruces} cruces")
    print(f"  Jugador 3: {j3_caras} caras, {j3_cruces} cruces\n")
    
    # Verificación de trampas
    print("Verifar trampas")
    if j1_caras == tiradas:
        print("Jugador 1 SIEMPRE saca cara")
    if j3_cruces == tiradas:
        print("Jugador 3 SIEMPRE saca cruz")
    print(f"Jugador 2 es honesto: {j2_caras/tiradas*100:.0f}% caras, {j2_cruces/tiradas*100:.0f}% cruces")

In [145]:
prog = Program(
    Declare('ro', 'BIT', 3),
    
    I(0),H(1),X(2),              

    MEASURE(0, ('ro', 0)),
    MEASURE(1, ('ro', 1)),
    MEASURE(2, ('ro', 2)),
).wrap_in_numshots_loop(50)

qvm = get_qc('3q-qvm')  # Ahora necesitamos 3 qubits
result = qvm.run(qvm.compile(prog)).get_register_map().get("ro")

es_ganador_3_jugadores(resultado=result, tiradas=50)

  Jugador 1: 50 caras, 0 cruces
  Jugador 2: 24 caras, 26 cruces
  Jugador 3: 0 caras, 50 cruces

Verifar trampas
Jugador 1 SIEMPRE saca cara (trampa exitosa)
Jugador 3 SIEMPRE saca cruz (trampa exitosa)
Jugador 2 es honesto: 48% caras, 52% cruces
